In [11]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.ensemble
import sklearn.metrics
import joblib
np.set_printoptions(precision=2, suppress=True)

In [2]:
class G:
    # global variables
    pass

In [3]:
G.df_train_full = pd.read_csv('X_train', dtype=np.int32)
G.df_test_full = pd.read_csv('X_test', dtype=np.int32)
G.y_train = np.array(open('Y_train').read().strip('\n').split('\n'),
                     dtype=np.int8)

In [6]:
def extract(df):
    c = len(df.columns)
    n = len(df)
    d = 1 + c + 5

    X = np.zeros((n, d), dtype=np.float64)
    X[:, 0] = 1  # bias
    X[:, 1:1+c] = df.values

    # quadratic term
    # ['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']
    idx = [1,2,4,5,6]
    X[:, 1+c:1+c+5] = X[:, idx] ** 2

    return X

def preprocess(df_train, df_test):
    X_train = extract(df_train)
    X_test = extract(df_test)
    X_full = np.concatenate((X_train, X_test), axis=0)
    
    means = np.mean(X_full, axis=0)
    stds = np.std(X_full, axis=0)

    # normalize
    # ['age', 'fnlwgt', 'hours_per_week', 'capital_gain', 'capital_loss']
    idx = [1,2,4,5,6]
    X_train[:, idx] = (X_train[:, idx] - means[idx]) / stds[idx]
    X_test[:, idx] = (X_test[:, idx] - means[idx]) / stds[idx]

    return X_train, X_test, means, stds

G.X_train, G.X_test, G.means, G.stds = preprocess(G.df_train_full, G.df_test_full)
# np.savetxt('a.csv',G.X_train,fmt='%.2f',delimiter=',')

In [7]:
clf = sklearn.ensemble.GradientBoostingClassifier(random_state=0)
scores = sklearn.model_selection.cross_val_score(
    clf, G.X_train, G.y_train, cv=5, scoring=['accuracy', 'neg_log_loss'])
print(scores)

[0.86 0.86 0.87 0.87 0.87]


In [ ]:
clf.fit(G.X_train, G.y_train)

In [10]:
G.y_test = clf.predict(G.X_test)
df_pred = pd.DataFrame({
    'id': np.arange(1, len(G.X_test)+1),
    'label': G.y_test
})
df_pred.to_csv('submission.csv', index=False)
print(df_pred['label'].values[:100])

[0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0]


In [12]:
joblib.dump(clf, 'boost/1.pkl')

['boost/1.pkl']